<a href="https://colab.research.google.com/github/lucghe/DAMGNet/blob/main/butMakeItDanube_Copy_of_Stylegan2_ada_Custom_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <a
    href="https://youtu.be/dcb4Ckpkx2o"
    target="_blank"
    rel="noopener noreferrer">
        <img
        alt="Night Sky Latent Walk"
        width="350" height="350"
        src="https://github.com/ArthurFDLR/GANightSky/blob/main/.github/random_walk.gif?raw=true">
    </a>
</p>

# 🚀 StyleGan2-ADA for Google Colab



1.   [Install StyleGAN2-ADA on your Google Drive](#scrollTo=5YcUMPQp6ipP)
2.   [Train a custom model](#scrollTo=Ti11YiPAiQpb)
3.   [Generate images from pre-trained model](#scrollTo=f0A9ZNtferpk)
4.   [Latent space exploration](#scrollTo=5yG1UyHXXqsO)


## Install StyleGAN2-ADA on your Google Drive

StyleGAN2-ADA only works with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.


In [1]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Thu Jan  6 11:51:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

Then, mount your Drive to the Colab notebook: 

In [2]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Finally, run this cell to install StyleGAN2-ADA on your Drive. If you’ve already installed the repository, it will skip the installation process and only check for updates. If you haven’t installed it, it will install all the necessary files. Beside, **in**, **out**, **datasets** and **training** folders are generated for data storage. Everything will be available on your Google Drive in the folder **StyleGAN2-ADA** even after closing this Notebook.

In [3]:
stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada
project_path        = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada
!git config --global user.name "ArthurFDLR"
!git config --global user.email "arthfind@gmail.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

/content/drive/MyDrive/StyleGAN2-ADA


## Train a custom model

Once you have installed StyleGAN2-ADA on your Google Drive and set up the working directory, you can upload your training dataset images in the associated folder.

In [4]:
dataset_name = 'danube'
datasets_source_path = project_path / 'datasets' / 'source' / (dataset_name + '.zip')
if datasets_source_path.is_dir():
    print("Dataset ready for import.")
else:
    print('Upload your images dataset as {}'.format(datasets_source_path))

Upload your images dataset as /content/drive/MyDrive/StyleGAN2-ADA/datasets/source/danube.zip


Unfortunately, large datasets might exceed the Google Drive quota after a few training batches. Indeed, StyleGAN2 download datasets multiple times during training. You might have to import your dataset in the local storage session. However, large files cannot be copy/paste from Drive *(Input/Output error)*. 

Run this cell to download your zipped dataset from your Drive and unzip it in the local session.

In [36]:
local_dataset_path = content_path / 'dataset'
print("Importing dataset...")
%mkdir "{local_dataset_path}"
%cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
print("Zip file succesfuly imported")

Importing dataset...
mkdir: cannot create directory ‘/content/dataset’: File exists
Zip file succesfuly imported


In [5]:
local_dataset_path = content_path / 'dataset'
if not local_dataset_path.is_dir():
    print("Importing dataset...")
    %mkdir "{local_dataset_path}"
    %cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
    print("Zip file succesfuly imported")
else:
    print('Zip file allready imported')

import zipfile
with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path))
print('Extraction completed')

Zip file allready imported


FileNotFoundError: ignored

In [37]:
import zipfile
with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path))
print('Extraction completed')

Extraction completed


In [38]:
%cd /content/dataset/
%pwd
%ls

/content/dataset
Chemical_Status_GWB_Stamen_Watercolor_512x512_center.jpg
Chemical_Status_GWB_Stamen_Watercolor_512x512_left.jpg
Chemical_Status_GWB_Stamen_Watercolor_512x512_right.jpg
Continuity_Interruptions_Ecological_Prioritisation_Stamen_Watercolor_512x512_center.jpg
Continuity_Interruptions_Ecological_Prioritisation_Stamen_Watercolor_512x512_left.jpg
Continuity_Interruptions_Ecological_Prioritisation_Stamen_Watercolor_512x512_right.jpg
danube.zip
Ecoregions_Stamen_Watercolor_512x512_center.jpg
Ecoregions_Stamen_Watercolor_512x512_left.jpg
Fish_Migration_Improvements_by_2021_Stamen_Watercolor_512x512_center.jpg
Fish_Migration_Improvements_by_2021_Stamen_Watercolor_512x512_left.jpg
Fish_Migration_Improvements_by_2021_Stamen_Watercolor_512x512_right.jpg
Flood_Hazard_and_Flooding_Scenarios_Stamen_Watercolor_512x512_center.jpg
Flood_Hazard_and_Flooding_Scenarios_Stamen_Watercolor_512x512_left.jpg
Flood_Hazard_and_Flooding_Scenarios_Stamen_Watercolor_512x512_right.jpg
Heavily_Modified_

In [40]:
ls

Chemical_Status_GWB_Stamen_Watercolor_512x512_center.jpg
Chemical_Status_GWB_Stamen_Watercolor_512x512_left.jpg
Chemical_Status_GWB_Stamen_Watercolor_512x512_right.jpg
Continuity_Interruptions_Ecological_Prioritisation_Stamen_Watercolor_512x512_center.jpg
Continuity_Interruptions_Ecological_Prioritisation_Stamen_Watercolor_512x512_left.jpg
Continuity_Interruptions_Ecological_Prioritisation_Stamen_Watercolor_512x512_right.jpg
Ecoregions_Stamen_Watercolor_512x512_center.jpg
Ecoregions_Stamen_Watercolor_512x512_left.jpg
Fish_Migration_Improvements_by_2021_Stamen_Watercolor_512x512_center.jpg
Fish_Migration_Improvements_by_2021_Stamen_Watercolor_512x512_left.jpg
Fish_Migration_Improvements_by_2021_Stamen_Watercolor_512x512_right.jpg
Flood_Hazard_and_Flooding_Scenarios_Stamen_Watercolor_512x512_center.jpg
Flood_Hazard_and_Flooding_Scenarios_Stamen_Watercolor_512x512_left.jpg
Flood_Hazard_and_Flooding_Scenarios_Stamen_Watercolor_512x512_right.jpg
Heavily_Modified_and_Artificial_SWB_Stamen_Wa

### Convert dataset to .tfrecords

Next, we need to convert our image dataset to a format that StyleGAN2-ADA can read:`.tfrecords`.

This can take a while.

In [16]:
cd tfr/

/content/dataset/tfr


In [43]:
%mkdir trf/images

In [20]:
local_images_path = local_dataset_path / 'images'
local_dataset_path /= 'tfr'

if (local_dataset_path).is_dir():
    print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
    print('Delete current dataset folder ({}) to regenerate tfrecords.'.format(local_dataset_path))
else:
    %mkdir "{local_dataset_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
        "{local_dataset_path}" "{local_images_path}"

mkdir: cannot create directory ‘/content/dataset/tfr/tfr’: No such file or directory
Loading images from "/content/dataset/tfr/images"
Error: No input images found


In [44]:
!python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images "/content/dataset/trf/images" "/content/dataset"

Loading images from "/content/dataset"
Creating dataset "/content/dataset/trf/images"
/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 101 images.
Traceback (most recent call last):
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py", line 1249, in <module>
    execute_cmdline(sys.argv)
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py", line 1244, in execute_cmdline
    func(**vars(args))
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py", line 710, in create_from_images
    img = np.asarray(PIL.Image.open(image_filenames[order[idx]]))
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2843, in open
    fp = builtins.open(filename, "rb")
IsADirectoryError: [Errno 21] Is a directory: '/content/dataset/trf'


In [54]:
pwd


'/content/dataset/trf/images'

There are numerous arguments to tune the training of your model. To obtain nice results, you will certainly have to experiment. Here are the most popular parameters:


*   *mirror:* Should the images be mirrored vertically?
*   *mirrory:* Should the images be mirrored horizontally?
*   *snap:* How often should the model generate image samples and a network pickle (.pkl file)?
*   *resume:* Network pickle to resume training from?

To see all the options, run the following ```help``` cell.

Please note that Google Colab Pro gives access to V100 GPUs, which drastically decreases (~3x) processing time over P100 GPUs.

In [52]:
!python "{stylegan2_repo_path / 'train.py'}" --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:


#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = 'color'

resume_from = 'brecahad512'

!python "{stylegan2_repo_path / 'train.py'}" --outdir="/content/drive/MyDrive/StyleGAN2-ADA/training/danube" \
    --data="/content/dataset/trf/images" \
    --snap={snapshot_count} --augpipe={augs} \
    --mirror={mirrored} --mirrory={mirroredY} \
    --metrics={metric_list} #--dry-run

tcmalloc: large alloc 4294967296 bytes == 0x55b3f41d6000 @  0x7f986c230001 0x7f98694161af 0x7f986946cc23 0x7f986946da87 0x7f986950f823 0x55b3ec2bb46c 0x55b3ec2bb240 0x55b3ec32f627 0x55b3ec329ced 0x55b3ec2bd48c 0x55b3ec2fe159 0x55b3ec2fb0a4 0x55b3ec2bbd49 0x55b3ec32f94f 0x55b3ec3299ee 0x55b3ec1fbe2b 0x55b3ec32bfe4 0x55b3ec3299ee 0x55b3ec1fbe2b 0x55b3ec32bfe4 0x55b3ec329ced 0x55b3ec1fbe2b 0x55b3ec32bfe4 0x55b3ec2bcafa 0x55b3ec32a915 0x55b3ec3299ee 0x55b3ec3296f3 0x55b3ec3f34c2 0x55b3ec3f383d 0x55b3ec3f36e6 0x55b3ec3cb163
tcmalloc: large alloc 4294967296 bytes == 0x55b4f41d6000 @  0x7f986c22e1e7 0x7f98694160ce 0x7f986946ccf5 0x7f986946cf4f 0x7f986950f673 0x55b3ec2bb46c 0x55b3ec2bb240 0x55b3ec32f627 0x55b3ec3299ee 0x55b3ec2bcbda 0x55b3ec32b737 0x55b3ec3299ee 0x55b3ec2bcbda 0x55b3ec32b737 0x55b3ec3299ee 0x55b3ec2bcbda 0x55b3ec32b737 0x55b3ec2bcafa 0x55b3ec32a915 0x55b3ec3299ee 0x55b3ec2bcbda 0x55b3ec32ed00 0x55b3ec3299ee 0x55b3ec2bcbda 0x55b3ec32b737 0x55b3ec329ced 0x55b3ec2bd48c 0x55b3ec2f

## Generate images from pre-trained model

You can finally generate images using a pre-trained network once everything is set-up. You can naturally use [your own model once it is trained](#scrollTo=Ti11YiPAiQpb&uniqifier=1) or use the ones NVLab published on [their website](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/).

<p align="center">
    <img
    alt="Night Sky Latent Walk"
    width="450" height="300"
    src="https://github.com/ArthurFDLR/GANightSky/blob/main/.github/Random_Generation.png?raw=true">
</p>

In [57]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

usage: generate.py generate-images [-h] --network NETWORK_PKL --seeds SEEDS
                                   [--trunc TRUNCATION_PSI]
                                   [--class CLASS_IDX] [--create-grid]
                                   [--outdir DIR] [--save_vector] [--fixnoise]
                                   [--jpg_quality JPG_QUALITY]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --seeds SEEDS         List of random seeds
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --class CLASS_IDX     Class label (default: unconditional)
  --create-grid         Add flag to save the generated images in a grid
  --outdir DIR          Root directory for run results (default: out)
  --save_vector         also save vector in .npy format
  --fixnoise            generate images using fixed noise (more accurate for
                        interpolations)
  

In [60]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 6

#generation_from = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl'
generation_from = '/content/drive/MyDrive/StyleGAN2-ADA/training/danube/00000-images-mirror-auto1-bgc-resumeffhq1024/network-snapshot-000000.pkl'

!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir="{project_path / 'out'}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/danube/00000-images-mirror-auto1-bgc-resumeffhq1024/network-snapshot-000000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 5150 (0/6) ...
Generating image for seed 5151 (1/6) ...
Generating image for seed 5152 (2/6) ...
Generating image for seed 5153 (3/6) ...
Generating image for seed 5154 (4/6) ...
Generating image for seed 5155 (5/6) ...
Generating image grid...


## Latent space exploration

It is also possible to explore the latent space associated with our model and [generate videos like this one](https://youtu.be/dcb4Ckpkx2o).


In [61]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --help 

usage: generate.py generate-latent-walk [-h] --network NETWORK_PKL
                                        [--trunc TRUNCATION_PSI]
                                        [--walk-type WALK_TYPE]
                                        [--frames FRAMES] [--fps FRAMERATE]
                                        [--seeds SEEDS] [--npys NPYS]
                                        [--save_vector] [--diameter DIAMETER]
                                        [--start_seed START_SEED]
                                        [--outdir DIR]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --walk-type WALK_TYPE
                        Type of walk (default: line)
  --frames FRAMES       Frame count (default: 240
  --fps FRAMERATE       Starting value
  --seeds SEEDS         List of random seeds
  --npys NPYS       

In [64]:
from numpy import random
walk_types = ['line', 'sphere', 'circularloop']
latent_walk_path = project_path / 'out' / 'latent_walk'
if not latent_walk_path.is_dir():
    %mkdir "{latent_walk_path}"

#explored_network = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl'
explored_network = '/content/drive/MyDrive/StyleGAN2-ADA/training/danube/00000-images-mirror-auto1-bgc-resumeffhq1024/network-snapshot-000000.pkl'

seeds = [random.randint(10000) for i in range(10)]
print(','.join(map(str, seeds)))
print("Base seeds:", seeds)
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_path}" --trunc=0.7 --walk-type="{walk_types[2]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20}

5998,7229,2139,9498,305,7436,2043,3234,5551,4316
Base seeds: [5998, 7229, 2139, 9498, 305, 7436, 2043, 3234, 5551, 4316]
Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/danube/00000-images-mirror-auto1-bgc-resumeffhq1024/network-snapshot-000000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/200 ...
Generating image for step 1/200 ...
Generating image for step 2/200 ...
Generating image for step 3/200 ...
Generating image for step 4/200 ...
Generating image for step 5/200 ...
Generating image for step 6/200 ...
Generating image for step 7/200 ...
Generating image for step 8/200 ...
Generating image for step 9/200 ...
Generating image for step 10/200 ...
Generating image for step 11/200 ...
Generating image for step 12/200 ...
Generating image for step 13/200 ...
Generating image for step 14/200 ...
Generating image for step 15/200 ...
Generating ima

## While you wait ...

... learn more about Generative Adversarial Networks and StyleGAN2-ADA:

*   [This Night Sky Does Not Exist](https://arthurfindelair.com/thisnightskydoesnotexist/): Generation of images from a model created using this Notebook on Google Colab Pro.
*   [This **X** Does Not Exist](https://thisxdoesnotexist.com/): Collection of sites showing the power of GANs.
*   [Karras, Tero, et al. _Analyzing and Improving the Image Quality of StyleGAN._ CVPR 2020.](https://arxiv.org/pdf/2006.06676.pdf): Paper published for the release of StyleGAN2-ADA.
*   [Official implementation of StyleGAN2-ADA](https://github.com/NVlabs/stylegan2-ada)
*   [StyleGAN v2: notes on training and latent space exploration](https://towardsdatascience.com/stylegan-v2-notes-on-training-and-latent-space-exploration-e51cf96584b3): Interesting article from Toward Data Science